In [ ]:
!pip install trl
!pip intall transformers
!pip install pandas

In [ ]:
import random

import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
)
from trl import RewardTrainer
import pandas as pd

In [ ]:
DATA_PATH = "CarperAI/openai_summarize_comparisons"
prepared_dataset = ... # load a subset of dataset
prepared_dataset

In [ ]:
model_name = ... # load distil roberta base model

model = ... # load pre-trained model
tokenizer = ... # load the corresponding tokenizer


In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = ... # chosen prompts
    prompt_plus_rejected_response = ... # rejected prompts

    # Then tokenize these modified fields.
    tokens_chosen = ... # encode the chosen prompts
    tokens_rejected = ... # encode the rejected prompts

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

formatted_dataset = prepared_dataset.map(formatting_func)
formatted_dataset = formatted_dataset.train_test_split()

training_args = TrainingArguments(
    output_dir="./results/rm",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=500
)

trainer = ... # reawrd trainer initialization

trainer.train()